In [8]:
import numpy as np
import pandas as pd

# 샘플 데이터 생성
np.random.seed(0)

groupA = np.random.normal(loc=10, scale=2.0, size=50) # 평균 10, 표준편차 2
groupB = np.random.normal(loc=12, scale=2.0, size=50) # 평균 12, 표준편차 2
groupC = np.random.normal(loc=14, scale=2.0, size=50) # 평균 14, 표준편차 2

df = pd.DataFrame({'A': groupA, 'B': groupB, 'C': groupC})

df.mean()

A    10.281119
B    11.958114
C    14.513070
dtype: float64

In [14]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# 모수 검정 (parametric test)
# 목적 : 여러 그룹 간의 평균 차이가 통계적으로 유의미한지를 검정
# 가정 : 각 그룹은 정규분포를 따르고 분산이 같으며(등분산성) 독립된 표본이어야 한다.
# H₀ (귀무가설) : 모든 그룹의 모평균이 같다.
# H₁ (대립가설) : 적어도 하나의 그룹의 평균이 다르다.
# 정규성이나 등분산성이 안 맞는 경우엔 Kruskal-Wallis 등의 비모수 검정 사용

# 그룹 이름과 값을 하나의 DataFrame으로 통합
df_new = pd.DataFrame({
    'value': np.concatenate([groupA, groupB, groupC]),
    'group': ['A']*50 + ['B']*50 + ['C']*50
})

print(df_new)

         value group
0    13.528105     A
1    10.800314     A
2    11.957476     A
3    14.481786     A
4    13.735116     A
..         ...   ...
145  15.888959     C
146  12.174356     C
147  16.234033     C
148  11.368185     C
149  13.076831     C

[150 rows x 2 columns]


In [22]:
# OLS 모델 적합 (종속변수 ~ 독립변수)
model = ols('value ~ C(group)', data=df_new).fit()

# ANOVA 수행
# typ=1 : 순서에 의존
# typ=2 : 평균 중심화된 효과
# typ=3 : 독립 효과 (SPSS 방식과 유사)
anova_results = anova_lm(model, typ=1)
anova_results
# C(group)는 그룹 간 차이에 대한 효과를 나타냄
# sum_sq : 제곱합
# mean_sq : 평균 제곱합, sum_sq / df
# df : 자유도
# F : F 통계량, 그룹 간 변동이 그룹 내 변동보다 얼마나 큰지 비교 (F가 크면 통계적으로 유의)
# PR(>F): p-value (0.05보다 작으면 그룹 간 평균 차이가 유의함)

,df,sum_sq,mean_sq,F,PR(>F)
C(group),2.0,454.158726,227.079363,54.119975,2.438362e-18
Residual,147.0,616.790127,4.195851,NaN,NaN


In [24]:
anova_results['F'][0], anova_results['PR(>F)'][0]

(54.11997511564046, 2.4383623043908716e-18)

In [9]:
# scipy의 f_oneway 사용
from scipy.stats import f_oneway

stat, p_value = f_oneway(df['A'], df['B'], df['C'])
stat, p_value

(54.11997511564093, 2.4383623043902153e-18)